In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
# import pyximport; pyximport.install()
from PedestrianCrowding import Road
from multiprocessing import Pool, Manager
import itertools
import pandas as pd
import plot_init as pu
import csv

In [2]:
# matplotlib.rcParams.update(pu.params)
matplotlib.rcParams.update(pu.params)

plt.rcParams["figure.figsize"]

[4.0, 3.0]

In [3]:
# %%timeit
# np.random.seed(456)
# roadlen=200
# num_lanes=1
# road = Road(roadlen, num_lanes, vmax=5, alpha=1e-3, frac_bus=1/num_lanes, periodic=True, density=0.3, p_slow=0.)
# data = []
# T_trans = 2000
# T = 2000
# std = np.zeros((T, num_lanes, roadlen))
# std_p = np.zeros((T, num_lanes, roadlen))
# for i in range(T_trans+T):
# #     std[i] = road.road
# #     std_p[i] = road.pedestrian

#     road.timestep_parallel()
#     if i>=T_trans:
#         for veh in road.vehicle_array:
#             data.append({
#                 "lane": veh.lane,
#                 "pos": veh.pos,
#                 "vel": veh.vel,
#                 "id": id(veh),
#                 "marker": veh.marker,
#                 "time": i
#             })

In [ ]:
sim_time = 3000
trans_time = 1000
num_lanes = 1

def simulate(density, frac_bus, trial, alpha):
    roadlength = 500
    vmax = 5
    alpha = alpha
    frac_bus = frac_bus
    density = density
    p_slow = 0.1
    periodic = True
    throughputs = []
    road = Road(roadlength, num_lanes, vmax, alpha, 
                        frac_bus, periodic, density, p_slow)
    for t in range(sim_time+trans_time):
        road.timestep_parallel()
        if t >= trans_time:
            throughputs.append(road.throughput())
    res = {"throughput": np.mean(throughputs),
            "frac_bus": frac_bus,
            "density": road.get_density(),
            "trial": trial,
            "alpha": alpha, 
            "p_slow": p_slow}    
    return res

densities = np.arange(0.02, 1, 0.02)
densities = np.arange(0.2, 0.81, 0.1)

bus_fractions = np.linspace(0, 1/num_lanes, 11)
trials = range(50)
alphas = np.geomspace(1e-4, 1, 49)
# alphas = np.geomspace(1e-4, 1, 29)

def g(tup):
    return simulate(*tup)
p = Pool()


with open('../data/dataset_safe_decel.csv', 'w') as f:
    writer = csv.writer(f)
    lines = 0
    ## crossover data
    densities = np.arange(0.2, 0.81, 0.1)
    alphas = np.geomspace(1e-4, 1, 49)
    for result in p.imap_unordered(g, itertools.product(densities, [1], trials, alphas)):
        if lines == 0:
            writer.writerow(result.keys())
            lines += 1
        writer.writerow(result.values())
        
    ## FD data
    densities = np.arange(0.02, 1, 0.02)
    bus_fractions = [0,0.2,0.5]
    for result in p.imap_unordered(g, itertools.product(densities, bus_fractions, trials, [1e-3])):
        writer.writerow(result.values())
        
    ## bus fraction crossover data
    densities = np.arange(0.02, 0.6, 0.02)
    bus_fractions = [0.1,0.4,0.7]
    alphas = np.geomspace(1e-4, 1, 49)
    for result in p.imap_unordered(g, itertools.product(densities, bus_fractions, trials, alphas)):
        writer.writerow(result.values())

In [7]:
ls

 anim.mp4                 'Multilane Traffic.ipynb'   SPP2019.ipynb
 Crowding.ipynb            PAASE.ipynb                Stations.ipynb
 DataGenerator.ipynb       PeriodicStations.ipynb     std_comparison.py
 images.tar                plot_init.py               tests.ipynb
 ManuscriptFigures.ipynb   __pycache__/               Untitled.ipynb


In [ ]:
# sim_time = 1000
# trans_time = 500
# num_lanes = 1

# def simulate(density, frac_bus, trial, alpha):
#     roadlength = 100
#     vmax = 1
#     alpha = alpha
#     frac_bus = frac_bus
#     density = density
#     p_slow = 0.5
#     periodic = True
#     throughputs = []
#     road = Road(roadlength, num_lanes, vmax, alpha, 
#                         frac_bus, periodic, density, p_slow)
#     for t in range(sim_time+trans_time):
#         road.timestep()
#         if t >= trans_time:
#             throughputs.append(road.throughput())
#     return {"throughput": np.mean(throughputs),
#             "frac_bus": frac_bus,
#             "density": road.get_density(),
#             "trial": trial,
#             "alpha": alpha, 
#             "p_slow": p_slow}

# densities = np.arange(0.02, 1, 0.02)
# bus_fractions = [1]
# trials = range(50)
# alphas = [0.0]
# p = Pool()
# s = p.starmap(simulate, itertools.product(densities, bus_fractions, trials, alphas))
# df_parallel = pd.DataFrame(s)

In [ ]:
q = df_parallel.groupby("density").mean().reset_index()
q['speed'] = q['throughput']/q['density']
q.plot(x="density", y="speed", marker='o', ms=2)
plt.grid()

In [ ]:
# road = Road(100, 2, vmax=5, frac_bus=0.5, alpha=0)
# # road.road[-1, 50] = 2
# road.pedestrian[-1, 20] = 1

# fig, ax = plt.subplots(1, figsize=(10,1))
# im = ax.imshow(road.road, vmin=0, vmax=2)

# # animation function.  This is called sequentially
# def animate(i):
#     road.timestep()
#     im.set_data(road.road)
#     return im,

# # # call the animator.  blit=True means only re-draw the parts that have changed.
# anim = animation.FuncAnimation(fig, animate,
#                                frames=100, interval=200, blit=True)
# HTML(anim.to_html5_video())

In [ ]:
roadlength = 50
num_lanes = 1
road = Road(roadlength, num_lanes, vmax=5, alpha=1e-2, frac_bus=0.1/num_lanes, periodic=True, density=0.3, p_slow=0)

# road.road[:, 50] = 2
# road.pedestrian[-1, 20] = 1
my_cmap = matplotlib.cm.get_cmap('gray_r')
my_cmap.set_over('r')

fig, ax = plt.subplots(1, figsize=(5,num_lanes/roadlength*5), dpi=200)
im = ax.imshow(road.road, vmin=0, vmax=1, cmap=my_cmap)
ax.set_yticks([])
# animation function.  This is called sequentially
def animate(i):
    road.timestep()
    im.set_data(road.road)
    return im,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate,
                               frames=200, interval=200, blit=True)
HTML(anim.to_html5_video())
anim.save("anim.mp4")

In [ ]:
sim_time = 3000
trans_time = 500
num_lanes = 1

def simulate(density, frac_bus, trial, alpha):
    roadlength = 400
    vmax = 5
    alpha = alpha
    frac_bus = frac_bus
    density = density
    p_slow = 0
    periodic = True
    throughputs = []
    road = Road(roadlength, num_lanes, vmax, alpha, 
                        frac_bus, periodic, density, p_slow)
    for t in range(sim_time+trans_time):
        road.timestep()
        if t >= trans_time:
            throughputs.append(road.throughput())
    return {"throughput": np.mean(throughputs),
            "frac_bus": frac_bus,
            "density": road.get_density(),
            "trial": trial,
            "alpha": alpha, 
            "p_slow": 0}

densities = np.arange(0.02, 1, 0.02)
bus_fractions = np.linspace(0, 1/num_lanes, 5)
trials = range(50)
alphas = [1e-3, 1e-2, 1e-1]
# p = Pool()
# s = p.starmap(simulate, itertools.product(densities, bus_fractions, trials, alphas))
# df = pd.DataFrame(s)
# df.to_csv("throughput-density-1lane.csv")

In [ ]:
plt.rc('font', family='serif')
# df = pd.DataFrame(s)
# df = pd.read_csv("throughput-density.csv")
df = pd.read_csv("throughput-density-1lane_parallel.csv")
num_lanes = 1

fig, axes = plt.subplots(1,2, figsize=(6,9/4), dpi=300)
for (bus_frac, alpha), group in df.groupby(["frac_bus", "alpha"]):
    if alpha==1e-2:
        group = group.groupby(["density", "alpha","frac_bus"])["throughput"].mean().reset_index()
        axes[1].plot(group["density"], group["throughput"]/group["density"], marker='o', lw=1,label=r"$f_B=%.1f$"%bus_frac, ms=2)
        axes[0].plot(group["density"], group["throughput"], marker='o', lw=1,label=r"$f_B=%.1f$"%bus_frac, ms=2)
axes[0].grid()
axes[1].grid()
axes[1].legend()
# axes[0].set_xlabel("density")
# axes[1].set_xlabel("density")
# axes[0].set_ylabel("flow (veh/timestep)")
# axes[1].set_ylabel("speed")
axes[0].set_xlabel(r"$\rho$")
axes[1].set_xlabel(r"$\rho$")
axes[0].set_ylabel(r"$q$")
axes[1].set_ylabel(r"$v$")
fig.savefig("images/fundamental_diagram_busfraction.pdf", bbox_inches="tight")

In [ ]:
# plot_alphas.index(alpha)

The plot above shows the flow vs density plots for $\alpha=0.01$. For $f_B>0$, we observe two competing trends. There appears to be some critical value of density for which the velocity increases until it converges to the trend of the $f_B=0$ case.

# Spatiotemporal Diagrams

In [ ]:
%run -i std_comparison.py

In [ ]:
### roadlen = 400
num_lanes = 1
# np.random.seed(2882936862)
# randseed = np.random.get_state()
road = Road(roadlen, num_lanes, vmax=5, alpha=1, frac_bus=1/num_lanes, periodic=True, density=0.3, p_slow=0.)

T = 800
std = np.zeros((T, num_lanes, roadlen))
for i in range(T):
    std[i] = road.road
    road.timestep()

fig_width = 2
my_cmap = matplotlib.cm.get_cmap('gray_r')
my_cmap.set_over('r')
fig, axes = plt.subplots(1,num_lanes,figsize=(fig_width,fig_width/num_lanes*(T/roadlen)), dpi=400, sharey=True, sharex=True)
if num_lanes == 1:
    axes = [axes]
for i, ax in enumerate(axes):
    im = ax.imshow(1*(std[:,i,:]), origin="lower", cmap=my_cmap, vmax=1.001)
ax_par = fig.add_subplot(111, frameon=True, zorder=-1)
ax_par.set_xlabel("position")
ax_par.set_ylabel("time")
ax_par.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
ax.axis("off")

# Phase transitions in alpha

In [ ]:
sim_time = 3000
trans_time = 1000
num_lanes = 1

def simulate(density, frac_bus, trial, alpha):
    roadlength = 200
    vmax = 5
    alpha = alpha
    frac_bus = frac_bus
    density = density
    p_slow = 0
    periodic = True
    throughputs = []
    road = Road(roadlength, num_lanes, vmax, alpha, 
                        frac_bus, periodic, density, p_slow)
    for t in range(sim_time+trans_time):
        road.timestep_parallel()
        if t >= trans_time:
            throughputs.append(road.throughput())
    return {"throughput": np.mean(throughputs),
            "frac_bus": frac_bus,
            "density": road.get_density(),
            "trial": trial,
            "alpha": alpha, 
            "p_slow": 0}

densities = np.arange(0.02, 1, 0.02)
bus_fractions = np.linspace(0, 1/num_lanes, 5)
trials = range(50)
alphas = np.geomspace(1e-4, 1, 49)
p = Pool()
# s = p.starmap(simulate, itertools.product(densities, bus_fractions, trials, alphas))
# df = pd.DataFrame(s)
# df.to_csv("alpha_transitions_parallel.csv")

In [ ]:
plt.rc('font', family='serif')
# df = pd.DataFrame(s)
# df = pd.read_csv("alpha_transitions.csv")
# num_lanes = 1
df = pd.read_csv("alpha_transitions_parallel.csv")
df = df[df["frac_bus"]==1]
df['speed'] = df["throughput"]/df["density"]
pt = pd.pivot_table(df, values="throughput", index="alpha", columns="density")
ps = pd.pivot_table(df, values="speed", index="alpha", columns="density")

df = df[df["alpha"].isin(df["alpha"].unique()[::12])]
plt.figure(figsize=(4,3), dpi=300)
fig, axes = plt.subplots(1,2, figsize=(6,9/4), dpi=300)
for (alpha), group in df.groupby(["alpha"]):
    group = group.groupby(["density", "alpha"])["throughput", "speed"].mean().reset_index()
    axes[1].plot(group["density"], group["speed"], marker='o', lw=1,label=r"$\lambda=10^{%.0f}$"%np.log10(alpha), ms=2)
    axes[0].plot(group["density"], group["throughput"], marker='o', lw=1,label=r"$\lambda=10^{%.0f}$"%np.log10(alpha), ms=2)
axes[0].grid()
axes[1].grid()
axes[1].legend()
axes[0].set_xlabel(r"$\rho$")
axes[1].set_xlabel(r"$\rho$")
axes[0].set_ylabel(r"$q$")
axes[1].set_ylabel(r"$\bar{v}$")
fig.savefig("images/fundamental_diagram_alpha.pdf", bbox_inches="tight")

In [ ]:
df = pd.read_csv("alpha_transitions_parallel.csv")
df.alpha.unique()

In [ ]:
plt.imshow(pt)
plt.yticks(range(pt.index.size)[::10], np.log10(pt.index)[::10])
plt.xticks(range(pt.columns.size)[::10], [round(i,2) for i in pt.columns[::10]])
cax = plt.colorbar()
cax.set_label("throughput")

In [ ]:
plt.imshow(ps)
plt.yticks(range(pt.index.size)[::10], np.log10(pt.index)[::10])
plt.xticks(range(pt.columns.size)[::10], [round(i,2) for i in pt.columns[::10]])
cax = plt.colorbar()
cax.set_label("speed")

# Order Parameter

In [ ]:
# sim_time = 1000
# trans_time = 1000
# num_lanes = 1

# def simulate(density, frac_bus, trial, alpha):
#     roadlength = 200
#     vmax = 5
#     alpha = alpha
#     frac_bus = frac_bus
#     density = density
#     p_slow = 0
#     periodic = True
#     throughputs = []
#     op = []
#     road = Road(roadlength, num_lanes, vmax, alpha, 
#                         frac_bus, periodic, density, p_slow)
#     for t in range(sim_time+trans_time):
#         road.timestep()
#         if t >= trans_time:
#             throughputs.append(road.throughput())
#             op.append(np.sum(np.convolve(np.array(road.road)[0]>0, np.ones((2,)), mode='valid')//2))
#     return {"throughput": np.mean(throughputs),
#             "frac_bus": frac_bus,
#             "density": road.get_density(),
#             "trial": trial,
#             "alpha": alpha, 
#             "p_slow": 0,
#             "order_parameter": np.mean(op)
# }

# densities = np.arange(0.02, 1, 0.02)
# bus_fractions = np.linspace(0, 1/num_lanes, 5)
# trials = range(50)
# alphas = np.geomspace(1e-4, 1, 49)
# p = Pool()
# s = p.starmap(simulate, itertools.product(densities, bus_fractions, trials, alphas))
# df = pd.DataFrame(s)
# df.to_csv("order_parameter.csv")

In [ ]:
num_lanes = 1
df = pd.read_csv("order_parameter.csv")
# df = df[df["frac_bus"]==0.25]
# df['speed'] = df["throughput"]/df["density"]/200/num_lanes
op = pd.pivot_table(df, values="order_parameter", index="alpha", columns="density")

plt.imshow(op, cmap="viridis_r")
plt.yticks(range(op.index.size)[::10], np.log10(op.index)[::10])
plt.xticks(range(op.columns.size)[::10], [round(i,2) for i in op.columns[::10]])
cax = plt.colorbar()
cax.set_label("order parameter")

In [ ]:
fig, ax = plt.subplots(1,1)
# plt.rcParams['figure.figsize'] = 3, 3*3/4


start = 9
stop = -1
skip = 10
for i, data in enumerate(ps.values[:,start:stop:skip].T):
#     ax.plot(alphas/(densities[i*skip+start]), (data-data.min())/(data.max()-data.min()), label=r"$\rho=%.2f$"%(densities[i*skip+start]))
    ax.plot(alphas, data, label=r"$\rho=%.2f$"%(densities[i*skip+start]), marker='o', ms=2)

ax.set_xscale('log')
# ax.set_yscale('log')

ax.set_xlabel(r"$\lambda$")
ax.set_ylabel(r"$\bar{v}$")
ax.legend()
ax.grid()
ax.yaxis.set_major_locator(plt.MultipleLocator(1))

# plt.axvline(10**-2)
fig.savefig("images/lambda_crossover.pdf", bbox_inches="tight")



In [ ]:
!tar -czf images.tar images

In [ ]:
fig.get_figwidth()

In [ ]:
ps.values


In [ ]:
grad = np.gradient(ps.values, 0.02, axis=0)
plt.plot(ps.columns.values, alphas[np.argmax(grad, axis=0)], marker='o', ms=4, )
plt.xlabel(r"$\rho$")
plt.ylabel(r"$\lambda^*$")
plt.grid()
plt.yscale("log")

In [ ]:
ps.index.values